In [223]:
import pandas as pd
import numpy as np
import pickle as p

In [247]:
data_all = pd.read_csv('data_all.csv')

# Test with Subset of Data
np.random.seed(21)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=10, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

# # Test with All Data
# data_subset = data_all.copy()

In [248]:
import sys
sys.getsizeof(data_all)

216607760

In [249]:
sorted(item_subset)

[202, 1221, 1225, 2256, 2867, 3718, 3767, 3821, 4410, 7108]

### `/train`

In [250]:
import requests
import json

url = 'https://fillet.azurewebsites.net/train/'
data_subset.to_parquet('data_subset.parquet')

files = {'data':open('data_subset.parquet', 'rb')}

payload = {'cv_acc': True,
           'project_id': 9
           }

headers = {
           'Accept-Charset': 'UTF-8'
          }

# payload = json.dumps(payload)
r = requests.post(url, data=payload,  timeout=1200, files=files)

files['data'].close()

In [251]:
payload

{'cv_acc': True, 'project_id': 9}

In [252]:
r.status_code

200

In [253]:
r.json()['result']

'Success'

In [254]:
# CV Accuracies Returned
cv_accuracies = pd.DataFrame.from_dict((r.json()['cv_acc']))
cv_accuracies

ValueError: DataFrame constructor not properly called!

### `/optimize`

In [ ]:
url = 'http://localhost:5000/optimize/'

payload = {'project_id': 1,
           'constraints': [],
           'population': 100,
           'max_epoch': 200
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [ ]:
best_prices = r.json()['result']
best_prices

In [ ]:
import requests
import json

url = 'http://localhost:5000/predict/'
payload = {'prices': best_prices,
           'project_id': 1
           }
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }
payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

# What If Prediction
qty = r.json()['qty_estimates']

In [ ]:
qty

In [ ]:
# Calculate Revenue
revenue = 0
for item in best_prices.keys():
    revenue += (best_prices[item]*qty['Qty_'+item])
    
print(revenue)

### `/predict/`

In [255]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

print('Input Prices Quantities:')
a_input

Input Prices Quantities:


{'202': 1.8,
 '1221': 3.2,
 '1225': 2.65,
 '2256': 8.35,
 '2867': 5.55,
 '3718': 7.55,
 '3767': 10.5,
 '3821': 10.75,
 '4410': 5.1,
 '7108': 4.2}

In [257]:
# Predict Sales Qty and Compare to Actual

import requests
import json

url = 'https://fillet.azurewebsites.net/predict/'

payload = {'prices': a_input, 'project_id': 9}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)

r = requests.post(url, data=payload, headers=headers)
print(r.status_code)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()

200


,Estimated,Actual
Qty_1221,211.0,211.0
Qty_1225,24.0,10.0
Qty_202,2029.0,1861.0
Qty_2256,43.0,37.0
Qty_2867,500.0,398.0
Qty_3718,149.0,136.0
Qty_3767,160.0,169.0
Qty_3821,11.0,11.0
Qty_4410,202.0,268.0
Qty_7108,51.0,41.0
